In [1]:
from sklearn.linear_model import SGDClassifier, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor 
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
#from bertviz import head_view

model = AutoModelForMaskedLM.from_pretrained("rifkat/uztext_568Mb_Roberta_BPE")
tokenizer = AutoTokenizer.from_pretrained("rifkat/uztext_568Mb_Roberta_BPE")

fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [3]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
data = pd.read_excel('classuztex.xlsx', engine='openpyxl',header=None)

In [4]:
data = data.sample(frac=1).reset_index(drop=True)

In [5]:
data

,0,1
0,"LG компанияси 1,7 миллион долларлик 325 дюймли...",10
1,Captiva 5. Янги модел ҳақида батафсил .,10
2,Шаҳрам Ғиёсов нима сабабдан ҳаваскор боксга қа...,7
3,Соч тўкилиши қандай содир бўлади?.,5
4,Ўзбекистон Москвада Афғонистон юзасидан музока...,2
...,...,...
295,Чекиш таъсири остида 25дан ортиқ касаллик куча...,5
296,Рухшона турмуш ўртоғи билан ажрашганини маълум...,8
297,Telegram ва TikTok ишида узилишлар кузатилди .,10
298,Таниқли актёр Бельмондо оламдан ўтди.,8


In [7]:
tokenized = data[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))


In [8]:
def token_model(tokenized, model):
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    attention_mask = np.where(padded != 0, 1, 0)
    input_ids = torch.tensor(padded)  
    attention_mask = torch.tensor(attention_mask)

    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)
    
    target = last_hidden_states[0][:,0,:].numpy()
    return target

In [9]:
target = token_model(tokenized,model)

In [11]:
labels = data[1]
labels = np.array(labels)

In [12]:
train_features, test_features, train_labels, test_labels = train_test_split(target, labels, test_size = 0.3, random_state = 42)

In [13]:
train_labels

array([10,  6,  9,  5,  4,  9,  5,  3, 10,  1,  8,  1,  5, 10,  5,  8,  9,
        8,  7,  9,  1,  4,  2,  8,  2,  7, 10, 10, 10,  4,  7,  5,  1, 10,
        2,  9,  6, 10,  9,  4,  4,  7,  7,  3,  7,  3,  8,  4,  7,  7,  9,
        1,  5,  4,  1,  2,  3,  5,  5,  1,  8,  6,  8,  4,  6,  3,  4,  9,
        8,  3,  8,  5,  2, 10,  5,  4,  8, 10,  6,  6,  5,  4,  4,  6,  8,
        8,  6, 10,  5,  5,  9, 10, 10,  4,  1,  2,  2,  7,  9, 10,  7,  9,
        1,  7,  2,  6,  8, 10,  8,  7,  2,  4,  4, 10,  1,  1,  6,  9,  8,
        3,  7,  6, 10,  5,  5,  7,  5,  5,  7,  5,  1,  6,  3, 10,  7,  2,
        5,  8,  1,  3,  5,  9,  7,  4, 10,  9,  7,  2,  9,  1,  9,  6,  6,
        6,  7,  5,  7,  1,  8,  5, 10, 10,  3, 10,  5,  8,  6,  6,  4,  4,
        8,  2,  3,  3,  7,  1,  6,  1,  9,  4,  2,  3,  5,  4,  8,  4,  3,
        2,  3,  9, 10,  3,  4,  9,  1,  1,  4,  5,  9,  3,  7,  8,  1,  8,
        1, 10,  3,  1, 10,  8])

In [14]:
train_labels=np.array(train_labels,dtype=int)

In [15]:
y = np.array(train_labels).reshape((-1, ))
y_test = np.array(test_labels).reshape((-1, ))

In [16]:
from sklearn.metrics import accuracy_score,f1_score
# Takes in a model, trains the model, and evaluates the model on the test set
def fit_and_evaluate(model):
    
    # Train the model
    model.fit(train_features, train_labels)
    
    # Make predictions and evalute
    model_pred = model.predict(test_features)
    #model_mae = mae(test_labels, model_pred)
    model_acc = accuracy_score(test_labels, model_pred)
    # Return the performance metric
    return model_acc

In [17]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_features, train_labels)
knn.score(test_features,  test_labels.astype(int))

0.6111111111111112

In [18]:
SGD = SGDClassifier()
SGD.fit(train_features, train_labels)
SGD.score(test_features,  test_labels.astype(int))

0.6888888888888889

In [19]:
RFC = RandomForestClassifier()
RFC.fit(train_features, train_labels)
RFC.score(test_features,  test_labels.astype(int))


0.8333333333333334